# Chapter 12: The functools Module

The `functools` module provides higher-order functions and operations on callable objects.
It is one of Python's most important standard library modules for functional programming,
offering tools for partial application, memoization, reduction, and more.

## Key Concepts
- **`partial`**: Pre-fill function arguments to create specialized versions
- **`lru_cache` / `cache`**: Automatic memoization of function results
- **`reduce`**: Cumulative application of a function over a sequence
- **`total_ordering`**: Auto-generate comparison methods
- **`wraps`**: Preserve function metadata in decorators
- **`singledispatch`**: Generic functions with type-based dispatch

## Section 1: functools.partial

`partial` creates a new function with some arguments pre-filled. This is useful for
adapting functions to interfaces that expect fewer arguments.

In [ ]:
import functools

# partial pre-fills function arguments
# Example: create a binary string parser from int()
base2_log = functools.partial(int, base=2)
print(f"binary '1010' = {base2_log('1010')}")   # 10
print(f"binary '1111' = {base2_log('1111')}")   # 15
print(f"binary '11001' = {base2_log('11001')}") # 25

# Create a hex parser too
hex_to_int = functools.partial(int, base=16)
print(f"\nhex 'ff' = {hex_to_int('ff')}")
print(f"hex '1a' = {hex_to_int('1a')}")

# partial with positional arguments
def power(base: float, exponent: float) -> float:
    return base ** exponent

square = functools.partial(power, exponent=2)
cube = functools.partial(power, exponent=3)

print(f"\nsquare(5) = {square(5)}")
print(f"cube(3) = {cube(3)}")

# Inspect partial objects
print(f"\npartial func: {square.func.__name__}")
print(f"partial keywords: {square.keywords}")

## Section 2: Memoization with lru_cache and cache

Memoization stores the results of expensive function calls and returns the cached result
when the same inputs occur again. `lru_cache` provides an LRU (Least Recently Used) cache,
while `cache` (Python 3.9+) provides an unbounded cache.

In [ ]:
import functools

# lru_cache: memoize with bounded cache
call_count: int = 0

@functools.lru_cache(maxsize=None)  # maxsize=None means unbounded
def fibonacci(n: int) -> int:
    """Compute the nth Fibonacci number with memoization."""
    global call_count
    call_count += 1
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

result = fibonacci(10)
print(f"fibonacci(10) = {result}")
print(f"Function was called {call_count} times (11 unique values, each computed once)")

# Cache info shows hits, misses, and size
print(f"Cache info: {fibonacci.cache_info()}")

# Second call is free -- all values are cached
call_count = 0
result = fibonacci(10)
print(f"\nSecond call: fibonacci(10) = {result}")
print(f"Function was called {call_count} times (all cache hits)")
print(f"Cache info: {fibonacci.cache_info()}")

In [ ]:
import functools

# functools.cache (Python 3.9+): simpler unbounded cache
# Equivalent to lru_cache(maxsize=None) but cleaner syntax
@functools.cache
def expensive_computation(x: int, y: int) -> int:
    """Simulate an expensive computation."""
    print(f"  Computing {x} + {y}...")
    return x + y

print("First calls (cache misses):")
print(f"Result: {expensive_computation(1, 2)}")
print(f"Result: {expensive_computation(3, 4)}")

print("\nSecond calls (cache hits -- no 'Computing...' output):")
print(f"Result: {expensive_computation(1, 2)}")
print(f"Result: {expensive_computation(3, 4)}")

# Bounded cache example: only keep the last 2 results
@functools.lru_cache(maxsize=2)
def bounded_compute(x: int) -> int:
    print(f"  Computing for {x}...")
    return x * 10

print("\n--- Bounded LRU cache (maxsize=2) ---")
bounded_compute(1)  # miss
bounded_compute(2)  # miss
bounded_compute(1)  # hit
bounded_compute(3)  # miss, evicts 2
bounded_compute(2)  # miss again (was evicted)
print(f"Cache info: {bounded_compute.cache_info()}")

## Section 3: functools.reduce

`reduce` applies a two-argument function cumulatively to the items of a sequence,
reducing it to a single value. It is the functional equivalent of a loop that accumulates a result.

In [ ]:
import functools
import operator

# reduce applies a function cumulatively: f(f(f(a, b), c), d)
# Product of all numbers (1 * 2 * 3 * 4 * 5 = 120)
product = functools.reduce(operator.mul, [1, 2, 3, 4, 5])
print(f"Product of [1,2,3,4,5] = {product}")

# Sum without using built-in sum()
total = functools.reduce(operator.add, [10, 20, 30, 40])
print(f"Sum of [10,20,30,40] = {total}")

# With an initial value
total_with_init = functools.reduce(operator.add, [1, 2, 3], 100)
print(f"Sum of [1,2,3] starting from 100 = {total_with_init}")

# Find the maximum value
largest = functools.reduce(lambda a, b: a if a > b else b, [3, 1, 4, 1, 5, 9, 2, 6])
print(f"\nLargest value: {largest}")

# Flatten a list of lists
nested = [[1, 2], [3, 4], [5, 6]]
flat = functools.reduce(operator.add, nested)
print(f"Flattened {nested} = {flat}")

# Build a dictionary from pairs
pairs = [("a", 1), ("b", 2), ("c", 3)]
result = functools.reduce(lambda d, pair: {**d, pair[0]: pair[1]}, pairs, {})
print(f"Dict from pairs: {result}")

## Section 4: functools.total_ordering

`total_ordering` is a class decorator that automatically fills in missing comparison methods.
You only need to define `__eq__` and one of `__lt__`, `__le__`, `__gt__`, or `__ge__`.

In [ ]:
import functools

@functools.total_ordering
class Score:
    """A score with all comparison operators auto-generated."""

    def __init__(self, value: int) -> None:
        self.value = value

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Score):
            return NotImplemented
        return self.value == other.value

    def __lt__(self, other: "Score") -> bool:
        return self.value < other.value

    def __repr__(self) -> str:
        return f"Score({self.value})"

# Only __eq__ and __lt__ are defined, but all comparisons work
s5 = Score(5)
s10 = Score(10)
s5b = Score(5)

print(f"{s5} < {s10}:  {s5 < s10}")
print(f"{s10} > {s5}:  {s10 > s5}")    # auto-generated
print(f"{s5} <= {s5b}: {s5 <= s5b}")    # auto-generated
print(f"{s10} >= {s5}: {s10 >= s5}")    # auto-generated
print(f"{s5} == {s5b}: {s5 == s5b}")

# Sorting works automatically
scores = [Score(85), Score(92), Score(78), Score(95), Score(88)]
print(f"\nSorted: {sorted(scores)}")

## Section 5: functools.wraps

`wraps` is a decorator for decorators. It copies the metadata (`__name__`, `__doc__`, etc.)
from the wrapped function to the wrapper, preventing metadata loss.

In [ ]:
import functools
import time

# Without @wraps: metadata is lost
def bad_timer(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"  {func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

# With @wraps: metadata is preserved
def good_timer(func):
    @functools.wraps(func)  # <-- preserves metadata
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"  {func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

@bad_timer
def process_bad(data: list[int]) -> int:
    """Process a list and return its sum."""
    return sum(data)

@good_timer
def process_good(data: list[int]) -> int:
    """Process a list and return its sum."""
    return sum(data)

print("Without @wraps:")
print(f"  Name: {process_bad.__name__}")      # 'wrapper' (wrong!)
print(f"  Doc:  {process_bad.__doc__}")        # None (lost!)

print("\nWith @wraps:")
print(f"  Name: {process_good.__name__}")      # 'process_good' (correct)
print(f"  Doc:  {process_good.__doc__}")        # preserved

# The decorated function still works
print()
process_good(list(range(1000000)))

## Section 6: functools.singledispatch

`singledispatch` transforms a function into a generic function that dispatches on the type
of its first argument. This is Python's way of doing function overloading.

In [ ]:
import functools

# singledispatch: type-based function overloading
@functools.singledispatch
def format_value(value) -> str:
    """Format a value for display (default implementation)."""
    return f"Unknown: {value!r}"

@format_value.register(int)
def _(value: int) -> str:
    return f"Integer: {value:,d}"

@format_value.register(float)
def _(value: float) -> str:
    return f"Float: {value:.2f}"

@format_value.register(str)
def _(value: str) -> str:
    return f"String: '{value}' (length={len(value)})"

@format_value.register(list)
def _(value: list) -> str:
    return f"List with {len(value)} items: {value}"

# Dispatches based on the type of the first argument
print(format_value(1234567))
print(format_value(3.14159))
print(format_value("hello world"))
print(format_value([1, 2, 3]))
print(format_value({"key": "value"}))  # Falls back to default

# Check registered implementations
print(f"\nRegistered types: {list(format_value.registry.keys())}")

In [ ]:
# Practical example: memoized recursive computation
import functools
import time

# Without memoization: exponential time complexity
def fib_slow(n: int) -> int:
    if n < 2:
        return n
    return fib_slow(n - 1) + fib_slow(n - 2)

# With memoization: linear time complexity
@functools.lru_cache(maxsize=128)
def fib_fast(n: int) -> int:
    if n < 2:
        return n
    return fib_fast(n - 1) + fib_fast(n - 2)

# Compare performance
start = time.perf_counter()
result_slow = fib_slow(30)
time_slow = time.perf_counter() - start

start = time.perf_counter()
result_fast = fib_fast(30)
time_fast = time.perf_counter() - start

print(f"fib_slow(30) = {result_slow}, took {time_slow:.4f}s")
print(f"fib_fast(30) = {result_fast}, took {time_fast:.6f}s")
print(f"Speedup: {time_slow / time_fast:.0f}x faster with memoization")

# lru_cache can handle much larger inputs
print(f"\nfib_fast(100) = {fib_fast(100)}")
print(f"Cache info: {fib_fast.cache_info()}")

## Summary

### functools.partial
- Creates specialized versions of functions with pre-filled arguments
- Useful for adapting functions to expected interfaces (e.g., `key=` functions)

### functools.lru_cache / functools.cache
- `lru_cache(maxsize=N)`: Bounded LRU memoization cache
- `lru_cache(maxsize=None)` or `cache`: Unbounded memoization
- Turns exponential-time recursion into linear-time with memoization
- Use `.cache_info()` to inspect hit/miss statistics

### functools.reduce
- Cumulatively applies a binary function over a sequence
- Useful with `operator` module functions (`operator.mul`, `operator.add`)
- Supports an optional initial value

### functools.total_ordering
- Class decorator: define `__eq__` + one comparison, get all six
- Enables sorting and all comparison operators automatically

### functools.wraps
- Always use in decorators to preserve function metadata
- Copies `__name__`, `__doc__`, `__module__`, and other attributes

### functools.singledispatch
- Type-based function overloading (dispatch on first argument type)
- Register implementations with `@func.register(type)`
- Falls back to the base implementation for unregistered types